分割数据

In [4]:
import pandas as pd
def split_csv(file_path, chunk_size):
   # 读取CSV文件
   df = pd.read_csv(file_path, chunksize=chunk_size,encoding='gbk')
   for i, chunk in enumerate(df):
       # 保存拆分后的文件
       chunk.to_csv(f'transit_{i}.csv', index=False)
# 使用示例
split_csv('原始数据\homework_hw_daily_公交车0312_O_800m.csv', 5000) # 每5000行拆分一个文件


<>:9: SyntaxWarning: invalid escape sequence '\h'
<>:9: SyntaxWarning: invalid escape sequence '\h'
C:\Users\TEST1\AppData\Local\Temp\ipykernel_1749040\3779699251.py:9: SyntaxWarning: invalid escape sequence '\h'
  split_csv('原始数据\homework_hw_daily_公交车0312_O_800m.csv', 5000) # 每5000行拆分一个文件


调用百度api获取出行成本，此时获取的信息是json格式

In [8]:
import pandas as pd
import requests
import time
import logging
import json
from concurrent.futures import ThreadPoolExecutor
from tqdm import tqdm

# 配置参数
AK_LIST = ['Tt6YLZdwRHnBlCPNP6AtXneIvMyTEzEt'
]  # 百度API密钥列表
INPUT_CSV_PATH = "测试.csv"  # CSV文件路径
TRANSPORT_MODE = "transit"  # 交通方式：公交/地铁
SUBWAY_PRIORITY = 3  # 3=不坐地铁
COORD_TYPE = "wgs84"  # 坐标类型
MAX_WORKERS = 3  # 并发线程数

# 配置日志
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s'
)
logger = logging.getLogger(__name__)

def get_subway_route(row, ak_index=0):
    """获取单条OD对的地铁路线信息，返回完整的route[0]信息"""
    try:
        # 提取坐标
        o_lat = row['O_lat']
        o_lon = row['O_long']
        d_lat = row['D_lat']
        d_lon = row['D_long']
        
        # 构造API请求URL
        current_ak = AK_LIST[ak_index]
        api_url = (
            f"https://api.map.baidu.com/direction/v2/{TRANSPORT_MODE}?"
            f"origin={o_lat},{o_lon}&"
            f"destination={d_lat},{d_lon}&"
            f"coord_type={COORD_TYPE}&"
            f"ak={current_ak}&"
            f"tactics_incity={SUBWAY_PRIORITY}"
        )
        
        # 发送请求
        time.sleep(0.2)  # 控制请求频率
        response = requests.get(api_url, timeout=10)
        result = response.json()
        
        # 处理响应
        if result['status'] == 0:  # 请求成功
            routes = result['result']['routes']
            if not routes:  # 无可用路线
                return {
                    'date_dt': row['date_dt'],
                    'o_grid': row['o_grid'],
                    'd_grid': row['d_grid'],
                    'num_freq': row['num_freq'],
                    'num_freq_t': row['num_freq_t'],
                    'O_long': row['O_long'],
                    'O_lat': row['O_lat'],
                    'D_long': row['D_long'],
                    'D_lat': row['D_lat'],
                    'route_info': None,
                    'status': 'no_route',
                    'message': '无可用路线'
                }
            
            # 获取最优路线（第一条路线）
            route = routes[0]
            
            # 返回完整的route信息
            return {
                'date_dt': row['date_dt'],
                'o_grid': row['o_grid'],
                'd_grid': row['d_grid'],
                'num_freq': row['num_freq'],
                'num_freq_t': row['num_freq_t'],
                'O_long': row['O_long'],
                'O_lat': row['O_lat'],
                'D_long': row['D_long'],
                'D_lat': row['D_lat'],
                'route_info': json.dumps(route, ensure_ascii=False),  # 将route信息转为JSON字符串
                'status': 'success',
                'message': '成功'
            }
        
        # API密钥失效，尝试下一个
        elif result['status'] in [302, 401] and ak_index + 1 < len(AK_LIST):
            return get_subway_route(row, ak_index + 1)
        
        # 其他错误
        else:
            return {
                'date_dt': row['date_dt'],
                'o_grid': row['o_grid'],
                'd_grid': row['d_grid'],
                'num_freq': row['num_freq'],
                'num_freq_t': row['num_freq_t'],
                'O_long': row['O_long'],
                'O_lat': row['O_lat'],
                'D_long': row['D_long'],
                'D_lat': row['D_lat'],
                'route_info': None,
                'status': 'api_error',
                'AK': AK_LIST[ak_index],
                'message': result.get('message', '未知错误')
            }
            
    except Exception as e:
        return {
            'date_dt': row['date_dt'],
            'o_grid': row['o_grid'],
            'd_grid': row['d_grid'],
            'num_freq': row['num_freq'],
            'num_freq_t': row['num_freq_t'],
            'O_long': row['O_long'],
            'O_lat': row['O_lat'],
            'D_long': row['D_long'],
            'D_lat': row['D_lat'],
            'route_info': None,
            'status': 'error',
            'message': str(e)
        }

def process_data():
    """批量处理数据"""
    try:
        # 读取CSV
        df = pd.read_csv(INPUT_CSV_PATH)
        logger.info(f"读取CSV成功，共{len(df)}条数据")
        
        # 多线程处理
        results = []
        with ThreadPoolExecutor(max_workers=MAX_WORKERS) as executor:
            futures = [executor.submit(get_subway_route, row) 
                      for _, row in df.iterrows()]
            
            # 使用tqdm显示进度
            for future in tqdm(futures, desc="处理进度"):
                try:
                    result = future.result()
                    results.append(result)
                except Exception as e:
                    logger.error(f"处理失败：{str(e)}")
        
        # 保存结果
        result_df = pd.DataFrame(results)
        output_path = f"subway_route_info_{time.strftime('%Y%m%d_%H%M%S')}.csv"
        result_df.to_csv(output_path, index=False, encoding='utf-8-sig')
        
        # 输出统计
        success_count = len(result_df[result_df['status'] == 'success'])
        logger.info(f"处理完成：")
        logger.info(f"- 总数据：{len(result_df)}条")
        logger.info(f"- 成功数：{success_count}条")
        logger.info(f"- 失败数：{len(result_df) - success_count}条")
        logger.info(f"- 结果保存至：{output_path}")
        
        # 显示样例
        if success_count > 0:
            print("\n处理成功的样例数据：")
            success_samples = result_df[result_df['status'] == 'success'].head()
            print(success_samples[['o_grid', 'd_grid', 'status']])
            
            # 显示第一条成功记录的route_info内容
            first_success = result_df[result_df['status'] == 'success'].iloc[0]
            route_info = json.loads(first_success['route_info'])
            print("\n第一条成功记录的route_info内容：")
            print(json.dumps(route_info, indent=2, ensure_ascii=False))
        
        return result_df
    
    except Exception as e:
        logger.error(f"处理出错：{str(e)}")
        raise

if __name__ == "__main__":
    try:
        process_data()
    except Exception as e:
        logger.error(f"程序运行出错：{str(e)}")
        raise

2025-10-28 17:47:00,762 - INFO - 读取CSV成功，共12条数据
处理进度: 100%|██████████| 12/12 [00:01<00:00,  9.57it/s]
2025-10-28 17:47:02,022 - INFO - 处理完成：
2025-10-28 17:47:02,023 - INFO - - 总数据：12条
2025-10-28 17:47:02,023 - INFO - - 成功数：0条
2025-10-28 17:47:02,024 - INFO - - 失败数：12条
2025-10-28 17:47:02,024 - INFO - - 结果保存至：subway_route_info_20251028_174702.csv


处理json文件

In [ ]:

import pandas as pd
import json
import ast
from tqdm import tqdm


def extract_step_by_step_info(input_csv_path, output_csv_path, max_steps=15):
    """
    从route_info中按步骤提取交通方式信息
    
    Parameters:
    input_csv_path: 输入CSV文件路径
    output_csv_path: 输出CSV文件路径
    max_steps: 最大步骤数
    """
    # 读取CSV文件
    df = pd.read_csv(input_csv_path,encoding='gbk')
    print(f"读取数据成功，共{len(df)}条记录")
    
    # 准备存储提取的数据
    extracted_data = []

    # 地铁线
    lines = ["1号线", "2号线", "3号线", "4号线", "5号线","6号线","7号线","8号线","10号线","12号线", "14号线","城郊线","郑许线"]
    
    # 遍历每一行数据
    for index, row in tqdm(df.iterrows(), total=len(df), desc="处理数据"):
        try:
            info = json.loads(row['route_info'])
            # 基础信息
            result = {
                'o_grid': row['o_grid'],
                'd_grid': row['d_grid'],
                'status': row['status'],
                'message': row['message'],
                'num_freq': row['num_freq'],
                'num_freq_t': row['num_freq_t'],
                'O_long': row['O_long'],
                'O_lat': row['O_lat'],
                'D_long': row['D_long'],
                'D_lat': row['D_lat'],
                'price':info['price'],
                'duration':info['duration']
            }
            
            # 初始化步骤列
            for i in range(1, max_steps + 1):
                result[f'步骤{i}_方式'] = ''
                result[f'步骤{i}_距离'] = 0
                result[f'步骤{i}_时间'] = 0
            
            # 如果route_info不为空且不是None
            if pd.notna(row['route_info']) and row['route_info'] != 'None':
                route_info_str = row['route_info']
                try:
                    # 首先尝试用json.loads解析
                    route_info = json.loads(route_info_str)
                except json.JSONDecodeError:
                    try:
                        # 如果失败，尝试用ast.literal_eval
                        route_info = ast.literal_eval(route_info_str)
                    except Exception as e:
                        print(f"解析route_info时出错 (行 {index}): {str(e)}")
                        print(f"问题数据: {route_info_str}")
                        route_info = None
                
                if route_info:
                    # 处理步骤信息
                    steps = route_info.get('steps', [])
                    step_count = 0
                    
                    for step in steps:
                        # 处理嵌套步骤
                        if isinstance(step, list) and step:
                            step = step[0]
                        
                        step_count += 1
                        if step_count > max_steps:
                            break
                        step_duration = step.get('duration', 0)   
                        step_distance = step.get('distance', 0)
                        instruction = step.get('instructions', '')
                        # 安全提取车辆信息
                        vehicle_info = step.get('vehicle_info')
                        type = vehicle_info.get('type')
                        detail = {}
                        if pd.notna(vehicle_info) and isinstance(vehicle_info, dict):
                            temp_detail = vehicle_info.get('detail', {})
                            if isinstance(temp_detail, dict):
                                detail = temp_detail
                        
                        trans_type_code = detail.get('type')

                        # 解析交通方式
                        transport_type = '其它'
                        if type == 5:
                            if "换乘" in instruction:
                                transport_type = '换乘'
                            else:
                                transport_type = '步行'
                        elif type == 3 and (trans_type_code == 0 or trans_type_code==5 or trans_type_code == 6 or trans_type_code == 9) :
                            transport_type = '公交'
                        else:
                            transport_type = '其它'

                        

                        # 记录步骤信息
                        result[f'步骤{step_count}_方式'] = transport_type
                        result[f'步骤{step_count}_距离'] = step_distance
                        result[f'步骤{step_count}_时间'] = step_duration
            
            extracted_data.append(result)
            
        except Exception as e:
            print(f"处理行 {index} 时出错: {str(e)}")
            # 添加错误信息
            error_result = {
                'o_grid': row.get('o_grid', ''),
                'd_grid': row.get('d_grid', ''),
                'status': 'error',
                'message': str(e)
            }
            # 添加空的步骤信息
            for i in range(1, max_steps + 1):
                error_result[f'步骤{i}_方式'] = ''
                error_result[f'步骤{i}_距离'] = 0
                error_result[f'步骤{i}_时间'] = 0
            extracted_data.append(error_result)
    
    # 创建新的DataFrame
    result_df = pd.DataFrame(extracted_data)
    
    # 保存结果
    result_df.to_csv(output_csv_path, index=False, encoding='utf-8-sig')
    print(f"结果已保存到: {output_file}")
    
    return result_df

# 使用示例
if __name__ == "__main__":
    input_file = "结果数据\\transit.csv"  # 文件路径
    output_file = "step_by_step_transit.csv"  # 输出文件路径
    
    result_df = extract_step_by_step_info(input_file, output_file)
    
    # 显示前几行结果
    print("\n处理结果样例:")
    print(result_df.head())

读取数据成功，共62925条记录


处理数据: 100%|██████████| 62925/62925 [00:08<00:00, 7478.92it/s]


结果已保存到: step_by_step_transit.csv

处理结果样例:
    o_grid   d_grid   status message  num_freq  num_freq_t      O_long  \
0  ww0v6u8  ww0vfqs  success      成功         1           2  113.676224   
1  ww0v6u8  ww0vg48  success      成功         1           2  113.676224   
2  ww0v6u8  ww0vkr5  success      成功         1           2  113.676224   
3  ww0v6u8  ww0vscs  success      成功         1           2  113.676224   
4  ww0v6u8  ww0vsvt  success      成功         1           2  113.676224   

       O_lat      D_long      D_lat  ...  步骤12_时间  步骤13_方式 步骤13_距离  步骤13_时间  \
0  34.698257  113.659744  34.797134  ...        0                0        0   
1  34.698257  113.687210  34.775162  ...        0                0        0   
2  34.698257  113.746262  34.711990  ...        0                0        0   
3  34.698257  113.769608  34.725723  ...        0                0        0   
4  34.698257  113.770981  34.747696  ...        0                0        0   

   步骤14_方式 步骤14_距离  步骤14_时间  步骤15_方式 步

合并地铁距离

In [6]:

import chardet
import pandas as pd

# 从CSV文件中读取数据
file_path = '结果数据\\step_by_step_transit.csv'

# 检测文件编码
with open(file_path, 'rb') as file:
    raw_data = file.read(10000)
    result = chardet.detect(raw_data)
    encoding = result['encoding']

# 使用检测到的编码重新读取数据
df = pd.read_csv(file_path, encoding=encoding)

# 提取方式和距离列名
method_columns = [f'步骤{i}_方式' for i in range(1, 11)]
distance_columns = [f'步骤{i}_距离' for i in range(1, 11)]

# 筛选出方式列中值为地铁的距离数据
subway_distances = df[method_columns].eq('地铁').values * df[distance_columns].values
foot_distances = df[method_columns].eq('步行').values * df[distance_columns].values + df[method_columns].eq('换乘').values * df[distance_columns].values
transit_distances = df[method_columns].eq('公交').values * df[distance_columns].values
total_duration = df[[f'步骤{i}_时间' for i in range(1, 11)]].sum(axis=1)

df['total_duration'] = total_duration
# 计算每一行是地铁的距离总和
df['subway_length'] = subway_distances.sum(axis=1)
df['foot_length'] = foot_distances.sum(axis=1)
df['transit_length'] = transit_distances.sum(axis=1)
# 提取需要的列
new_df = df[['o_grid', 'd_grid', 'num_freq','num_freq_t','O_long','O_lat','D_long','D_lat','price','total_duration','subway_length','foot_length','transit_length']]

# 将结果保存为 CSV 文件
csv_path = '公交出行成本.csv'
new_df.to_csv(csv_path, index=False)

C:\Users\TEST1\AppData\Local\Temp\ipykernel_1756356\579558181.py:14: DtypeWarning: Columns (39,42) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path, encoding=encoding)
